# Analysis

## Possible Methods
- Pixel-wise MSE
- Peak signal-to-noise ratio
- Structural similarity index
- Mean-Absolute Error
- LPIPS
- Frechet Inception Distance

In [1]:
# Define inputs and outputs
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
im_input = Image.open("./images/input/bball.jpg")
im_output = Image.open("./output/result_1200.png")

# resize input to match output
im_input = im_input.resize(im_output.size)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\mrp12\\Documents\\cs445-final-project\\images\\input\\bball.jpg'

In [3]:
# Mean Squared Error
def MSE(im_input, im_output):
    N = im_input.size[0]*im_input.size[1]*len(im_input.getbands()) # h*w*channels
    error = np.subtract(im_input, im_output)
    square_error = np.square(error)
    sum_square_error = np.sum(square_error)
    mean_squared_error = sum_square_error / N
    return mean_squared_error

# Mean Absolute Error
def MAE(im_input, im_output):
    N = im_input.size[0]*im_input.size[1]*len(im_input.getbands()) # h*w*channels
    error = np.subtract(im_input, im_output)
    sum_error = np.sum(error)
    mean_absolute_error = sum_error / N
    return mean_absolute_error

# MSE(im_input, im_output)

In [4]:
# Peak Signal-to-Noise Ratio
def PSNR(im_input, im_output):
    im_mode = im_input.mode
    # modes info from PIL documentation
    max_intensity = 255
    if im_mode == 1 or im_mode == 'F':
        max_intensity = 1
    if im_mode == 'I':
        # from np.iinfo(np.int32).max
        max_intensity = 2147483647
    mse = MSE(im_input, im_output)
    psnr = 20*np.log10(max_intensity) - 10*np.log10(mse)
    return psnr
    
# PSNR(im_input, im_output)

In [6]:
# Structural Similarity Index, https://en.wikipedia.org/wiki/Structural_similarity_index_measure
def SSIM(im_input, im_output):
    return (
        SSIM_LUMINANCE(im_input, im_output),
        SSIM_CONTRAST(im_input, im_output),
        SSIM_STRUCTURE(im_input, im_output),
        )

# luminance component
def SSIM_LUMINANCE(im_input, im_output):
    N = im_input.size[0]*im_input.size[1]*len(im_input.getbands()) # h*w*channels
    # pixel sample means
    mu_input = np.average(im_input)
    mu_output = np.average(im_output)
    # variances
    
    # dynamic range
    im_mode = im_input.mode
    # modes info from PIL documentation
    L = 255
    if im_mode == 1 or im_mode == 'F':
        L = 1
    if im_mode == 'I':
        # from np.iinfo(np.int32).max
        L = 2147483647
    
    # c1, c2 prevents divide by zero issues
    k1 = 0.01 # per SSIM wikipedia
    c1 = (k1*L)**2
    
    lum = (2*mu_input*mu_output+c1)
    lum /= (mu_input**2 + mu_output**2 + c1)
    return lum

# contrast component
def SSIM_CONTRAST(im_input, im_output):
    # variances
    var_input = np.var(im_input)
    var_output = np.var(im_output)
    
    # dynamic range
    im_mode = im_input.mode
    # modes info from PIL documentation
    L = 255
    if im_mode == 1 or im_mode == 'F':
        L = 1
    if im_mode == 'I':
        # from np.iinfo(np.int32).max
        L = 2147483647
    
    # c1, c2 prevents divide by zero issues
    k2 = 0.03 # per SSIM wikipedia
    c2 = (k2*L)**2
    
    cont = 2*np.sqrt(var_input)*np.sqrt(var_output) + c2
    cont /= (var_input + var_output + c2)
    return cont

# structure component
def SSIM_STRUCTURE(im_input, im_output):
    N = im_input.size[0]*im_input.size[1]*len(im_input.getbands()) # h*w*channels
    # pixel sample means
    mu_input = np.average(im_input)
    mu_output = np.average(im_output)
    # variances
    var_input = np.var(im_input)
    var_output = np.var(im_output)
    # covariance = sum[(xi-xbar)*(yi-ybar)]/(N-1)
    covariance = np.sum((im_input-mu_input)*(im_output-mu_output))/(N-1)
    
    # dynamic range
    im_mode = im_input.mode
    # modes info from PIL documentation
    L = 255
    if im_mode == 1 or im_mode == 'F':
        L = 1
    if im_mode == 'I':
        # from np.iinfo(np.int32).max
        L = 2147483647
    
    # c1, c2 prevents divide by zero issues
    k2 = 0.03 # per SSIM wikipedia
    c2 = (k2*L)**2
    c3 = c2/2
    
    struc = covariance + c3
    struc /= np.sqrt(var_input)*np.sqrt(var_output) + c3
    return struc

# display(SSIM(im_input, im_output))

In [37]:
# Define inputs and outputs
from PIL import Image
import numpy as np
# im_input = Image.open("./images/input/bball.jpg")
# im_output = Image.open("./output/result_1200.png")
# # resize input to match output
# im_input = im_input.resize(im_output.size)


import os

input_directory = "resources/input"
style_directory = "resources/style"
output_directory = "resources/output"

output_ext = "_500.png"

N_inputs = len(os.listdir(input_directory))
N_styles = len(os.listdir(style_directory))

# input image name, style image name, MSE, MAE, PSNR, SSIM_L, SSIM_C, SSIM_S
results = np.zeros((N_inputs*N_styles, 8), dtype=object)

count = 0
for input_idx in range(N_inputs):
    for style_idx  in range(N_styles):
        input_src = input_directory + "/" + os.listdir(input_directory)[input_idx]
        # https://stackoverflow.com/questions/678236/how-do-i-get-the-filename-without-the-extension-from-a-path-in-python
        output_src = output_directory + "/" + os.path.splitext(os.path.basename(os.listdir(style_directory)[style_idx]))[0] + "/" + os.listdir(input_directory)[input_idx] + output_ext
        im_input = Image.open(input_src)
        im_output = Image.open(output_src)
        im_input = im_input.resize(im_output.size)
        input_image_name = os.path.splitext(os.path.basename(input_src))[0]
        style_image_name = os.path.splitext(os.listdir(style_directory)[style_idx])[0]
        val_mse = str(MSE(im_input, im_output))
        val_mae = str(MAE(im_input, im_output))
        val_psnr = str(PSNR(im_input, im_output))
        val_ssim_l = str(SSIM_LUMINANCE(im_input, im_output))
        val_ssim_c = str(SSIM_CONTRAST(im_input, im_output))
        val_ssim_s = str(SSIM_STRUCTURE(im_input, im_output))
        results[count,:] = [input_image_name, style_image_name, val_mse, val_mae, val_psnr, val_ssim_l, val_ssim_c, val_ssim_s]
        count += 1

# https://www.geeksforgeeks.org/how-to-create-a-csv-file-using-python/
import csv
with open("analysis.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(results.tolist())

# for style_folder in os.listdir(directory):
#     folder_path = os.path.join(directory, style_folder)
#     if os.path.isdir(folder_path):
#         for output_file in os.listdir(folder_path):
#             output_path = os.path.join(folder_path, output_file)
#             if os.path.isfile(output_path):
                

In [44]:
# CycleGAN Analysis

# Define inputs and outputs
from PIL import Image
import numpy as np
# im_input = Image.open("./images/input/bball.jpg")
# im_output = Image.open("./output/result_1200.png")
# # resize input to match output
# im_input = im_input.resize(im_output.size)


import os

input_directory = "resources/input-gan"
output_directory = "resources/gan"

output_ext = "_AtoB.jpg"

N_inputs = len(os.listdir(input_directory))

# input image name, style image name, MSE, MAE, PSNR, SSIM_L, SSIM_C, SSIM_S
results = np.zeros((N_inputs, 8), dtype=object)

count = 0
for input_idx in range(N_inputs):
    input_src = input_directory + "/" + os.listdir(input_directory)[input_idx]
    # https://stackoverflow.com/questions/678236/how-do-i-get-the-filename-without-the-extension-from-a-path-in-python
    output_src = output_directory + "/"+ os.path.splitext(os.path.basename(os.listdir(input_directory)[input_idx]))[0] + output_ext
    im_input = Image.open(input_src)
    im_output = Image.open(output_src)
    im_input = im_input.resize(im_output.size)
    input_image_name = os.path.splitext(os.path.basename(input_src))[0]
    style_image_name = "GAN"
    val_mse = str(MSE(im_input, im_output))
    val_mae = str(MAE(im_input, im_output))
    val_psnr = str(PSNR(im_input, im_output))
    val_ssim_l = str(SSIM_LUMINANCE(im_input, im_output))
    val_ssim_c = str(SSIM_CONTRAST(im_input, im_output))
    val_ssim_s = str(SSIM_STRUCTURE(im_input, im_output))
    results[count,:] = [input_image_name, style_image_name, val_mse, val_mae, val_psnr, val_ssim_l, val_ssim_c, val_ssim_s]
    count += 1

# https://www.geeksforgeeks.org/how-to-create-a-csv-file-using-python/
import csv
with open("analysis-gan.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(results.tolist())

# for style_folder in os.listdir(directory):
#     folder_path = os.path.join(directory, style_folder)
#     if os.path.isdir(folder_path):
#         for output_file in os.listdir(folder_path):
#             output_path = os.path.join(folder_path, output_file)
#             if os.path.isfile(output_path):
                

In [ ]:
# Texture Transfer Analysis

# Define inputs and outputs
from PIL import Image
import numpy as np
# im_input = Image.open("./images/input/bball.jpg")
# im_output = Image.open("./output/result_1200.png")
# # resize input to match output
# im_input = im_input.resize(im_output.size)


import os

input_directory = "resources/input-gan"
output_directory = "resources/texture-transfer"

output_ext = "-tt.jpg"

N_inputs = len(os.listdir(input_directory))

# input image name, style image name, MSE, MAE, PSNR, SSIM_L, SSIM_C, SSIM_S
results = np.zeros((N_inputs, 8), dtype=object)

count = 0
for input_idx in range(N_inputs):
    input_src = input_directory + "/" + os.listdir(input_directory)[input_idx]
    # https://stackoverflow.com/questions/678236/how-do-i-get-the-filename-without-the-extension-from-a-path-in-python
    output_src = output_directory + "/"+ os.path.splitext(os.path.basename(os.listdir(input_directory)[input_idx]))[0] + output_ext
    im_input = Image.open(input_src)
    im_output = Image.open(output_src)
    im_input = im_input.resize(im_output.size)
    input_image_name = os.path.splitext(os.path.basename(input_src))[0]
    style_image_name = "bazille_and_camille_(study_for__dejeuner_sur_l'herbe_)_1970.17.41"
    val_mse = str(MSE(im_input, im_output))
    val_mae = str(MAE(im_input, im_output))
    val_psnr = str(PSNR(im_input, im_output))
    val_ssim_l = str(SSIM_LUMINANCE(im_input, im_output))
    val_ssim_c = str(SSIM_CONTRAST(im_input, im_output))
    val_ssim_s = str(SSIM_STRUCTURE(im_input, im_output))
    results[count,:] = [input_image_name, style_image_name, val_mse, val_mae, val_psnr, val_ssim_l, val_ssim_c, val_ssim_s]
    count += 1

# https://www.geeksforgeeks.org/how-to-create-a-csv-file-using-python/
import csv
with open("analysis-gan.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(results.tolist())

# for style_folder in os.listdir(directory):
#     folder_path = os.path.join(directory, style_folder)
#     if os.path.isdir(folder_path):
#         for output_file in os.listdir(folder_path):
#             output_path = os.path.join(folder_path, output_file)
#             if os.path.isfile(output_path):
                